# Fraud Detection in transactions with Keras

### Trying to build a suitable NN with Keras, dealing with the number of numerical and categorical variables 


In [ ]:
#  Libraries
import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
from sklearn import preprocessing
# Suppr warning
import warnings
warnings.filterwarnings("ignore")

import itertools
from scipy import interp
# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

## Data Loading
Just the standard loading of the data used in most other kernels. 

In [ ]:

train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')
train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')
sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

In [ ]:
train_df = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test_df = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
print("Train set shape : "+str(train_df.shape))
print("Test set shape  : "+str(test_df.shape))

#### Dropping TransactionDT (time) : we don't want to learn from this

In [ ]:
train_df = train_df.drop(["TransactionDT"], axis = 1).reset_index()
test_df = test_df.drop(["TransactionDT"], axis = 1).reset_index()

In [ ]:
train_df['nul'] = train_df.isna().sum(axis=1)
test_df['nul'] = test_df.isna().sum(axis=1)

In [ ]:
train_df.head()

#### Dropping very sparse categoricals features like deviceinfo and deviceid. We keep only ~55 : after some tests, a lot of variables cause rapid overfitting...

In [ ]:
train_df_ = train_df.iloc[:, :55]
test_df_ = test_df.iloc[:, :54]

In [ ]:
del train_transaction, train_identity, test_transaction, test_identity

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']
#https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest_df-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    train_df[c + '_bin'] = train_df[c].map(emails)
    test_df[c + '_bin'] = test_df[c].map(emails)
    
    train_df[c + '_suffix'] = train_df[c].map(lambda x: str(x).split('.')[-1])
    test_df[c + '_suffix'] = test_df[c].map(lambda x: str(x).split('.')[-1])
    
    train_df[c + '_suffix'] = train_df[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test_df[c + '_suffix'] = test_df[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [ ]:
for c1, c2 in train_df.dtypes.reset_index().values:
    if c2=='O':
        train_df[c1] = train_df[c1].map(lambda x: str(x).lower())
        test_df[c1] = test_df[c1].map(lambda x: str(x).lower())

## Numerical and Categorical
List the cat and num variables

In [ ]:
numerical = ["TransactionAmt", "nulls1", "dist1", "dist2"] + \
            ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] + \
            ["V" + str(i) for i in range(1, 340)]
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain_bin", "P_emaildomain_suffix", "R_emaildomain_bin", "R_emaildomain_suffix",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]


In [ ]:
numerical = [col for col in numerical if col in train_df.columns]
categorical = [col for col in categorical if col in train_df.columns]

### Deal with NaN:
We could fill with the mean of the trainSet and then plug the train mean in the testSet, but here fore simplicity I fill with zeros

In [ ]:
def dealWithNan(df):
    for x in list(df.columns.values):
        if x in numerical:
            #print("___________________"+x)
            #print(df[x].isna().sum())
            df[x] = df[x].fillna(0)
           #print("Mean-"+str(df[x].mean()))
    return df
train_df=dealWithNan(train_df)
test_df=dealWithNan(test_df)

## Label Encoding of the cat variables


In [ ]:
# Label Encoding
category_counts = {}
for f in categorical:
    train_df[f] = train_df[f].replace("nan", "other")
    train_df[f] = train_df[f].replace(np.nan, "other")
    test_df[f] = test_df[f].replace("nan", "other")
    test_df[f] = test_df[f].replace(np.nan, "other")
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[f].values) + list(test_df[f].values))
    train_df[f] = lbl.transform(list(train_df[f].values))
    test_df[f] = lbl.transform(list(test_df[f].values))
    category_counts[f] = len(list(lbl.classes_)) + 1


In [ ]:
from sklearn.preprocessing import StandardScaler

## Numerical Scaling

Scaling of the numerical data , better with a NN. 
First we scale down the extreme positiv variables with a log1p transformation

Then, standard scaler. Better for NN convergence

In [ ]:
for column in numerical:
    scaler = StandardScaler()
    if train_df[column].max() > 100 and train_df[column].min() >= 0:
        train_df[column] = np.log1p(train_df[column])
        test_df[column] = np.log1p(test_df[column])
    scaler.fit(np.concatenate([train_df[column].values.reshape(-1,1), test_df[column].values.reshape(-1,1)]))
    train_df[column] = scaler.transform(train_df[column].values.reshape(-1,1))
    test_df[column] = scaler.transform(test_df[column].values.reshape(-1,1))

In [ ]:
target = 'isFraud'

#### Train Val Split

In [ ]:
tr_df, val_df = train_test_split(train_df, test_size = 0.2, random_state = 42, shuffle = False)

Grabbing the features we want to pass into the neural network

In [ ]:
def get_input_features(df):
    X = {'numerical':np.array(df[numerical])}
    for cat in categorical:
        X[cat] = np.array(df[cat])
    return X

## Neural Network Model 

We will use the embedding layer for categoricals and the numericals will go through feed forward dense layers. 

We create our embedding layers such that we have as many rows as we had categories and the dimension of the embedding is the log1p + 1 of the number of categories : the log prevents to have to many dimensions for the categorical variables with a high cardinality.

We will then pass the embeddings through a spatial dropout layer which will drop dimensions within the embedding across batches and then flatten and concatenate. Then we will concatenate this to the numerical features and apply batch norm and then add some more dense layers after. 

In [ ]:
from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k
def nn_model():
    k.clear_session()

    categorical_inputs = []
    for cat in categorical:
        categorical_inputs.append(Input(shape=[1], name=cat))

    categorical_embeddings = []
    for i, cat in enumerate(categorical):
        categorical_embeddings.append(
            Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + "_embed")(categorical_inputs[i]))

    categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
#     categorical_logits = Dropout(.5)(categorical_logits)

    numerical_inputs = Input(shape=[tr_df[numerical].shape[1]], name = 'numerical')
    numerical_logits = Dropout(.1)(numerical_inputs)
  
    x = Concatenate()([
        categorical_logits, 
        numerical_logits,
    ])
#     x = categorical_logits
#     x = BatchNormalization()(x)
    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.2)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.2)(x)
    out = Dense(1, activation = 'sigmoid')(x)
    

    model = Model(inputs=categorical_inputs + [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.01), loss = loss)
    return model


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from copy import deepcopy
categorical_save = deepcopy(categorical)
numerical_save = deepcopy(numerical)

# Greedy Feature Selection

First we will train the NN with all features in order to make a reference

After that we greedily drop one feature at a time and see if it increases or decreases performance. If it increases upon dropping the feature then we will drop the feature. 

We will iterate through epochs of the model and save the model weights if the score is an improvement upon previous best roc_auc_scores since this is competition metric. If the NN does not improve upon previous best after 4 epochs we will skip the rest of the training steps to save time. 

In [ ]:
#in case we want to restart the search. uncomment and rerun
# categorical = deepcopy(categorical_save)
# numerical = deepcopy(numerical_save)

In [ ]:
X_train = get_input_features(tr_df)
X_valid = get_input_features(val_df)
X_test = get_input_features(test_df)
y_train = tr_df[target]
y_valid = val_df[target]
model = nn_model()
best_score = 0
patience = 0
for i in range(100):
    if patience < 3:
        hist = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), batch_size = 8000, epochs = 1, verbose = 1)
        valid_preds = model.predict(X_valid, batch_size = 8000, verbose = True)
        score = roc_auc_score(y_valid, valid_preds)
        print(score)
        if score > best_score:
            model.save_weights("model.h5")
            best_score = score
            patience = 0
        else:
            patience += 1

### Dropping categoricals

In [ ]:
drop_cats = []

In [ ]:
for category_test in categorical_save:
    categorical = [cat for cat in categorical_save if cat not in drop_cats]
    categorical.remove(category_test)
    print(categorical)
    X_train = get_input_features(tr_df)
    X_valid = get_input_features(val_df)
    X_test = get_input_features(test_df)
    y_train = tr_df[target]
    y_valid = val_df[target]
    model = nn_model()
    local_score = 0
    patience = 0
    for i in range(100):
        if patience < 3:
            hist = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), batch_size = 8000, epochs = 1, verbose = 0)
            valid_preds = model.predict(X_valid, batch_size = 8000, verbose = False)
            score = roc_auc_score(y_valid, valid_preds)
            if score > local_score:
                model.save_weights("model.h5")
                local_score = score
                patience = 0
            else:
                patience += 1
        else:
            if local_score < best_score:
                print("performance reduced when", category_test, "dropped to", local_score, "from", best_score)
                break
            else:
                drop_cats.append(category_test)
                print("performance increased when", category_test, "dropped to", local_score, "from", best_score)
                best_score = local_score
                model.save_weights("best_model.h5")
                break

In [ ]:
drop_cats

In [ ]:
categorical = [cat for cat in categorical_save if cat not in drop_cats]

In [ ]:
drop_nums = []

### Dropping numeric

In [ ]:
for numerical_test in numerical_save:
    numerical = [num for num in numerical_save if num not in drop_nums]
    numerical.remove(numerical_test)
    print(numerical)
    X_train = get_input_features(tr_df)
    X_valid = get_input_features(val_df)
    X_test = get_input_features(test_df)
    y_train = tr_df[target]
    y_valid = val_df[target]
    model = nn_model()
    local_score = 0
    patience = 0
    for i in range(100):
        if patience < 4:
            hist = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), batch_size = 8000, epochs = 1, verbose = 0)
            valid_preds = model.predict(X_valid, batch_size = 8000, verbose = False)
            score = roc_auc_score(y_valid, valid_preds)
            if score > local_score:
                model.save_weights("model.h5")
                local_score = score
                patience = 0
            else:
                patience += 1
        else:
            if local_score < best_score:
                print("performance reduced when", numerical_test, "dropped to", local_score, "from", best_score)
                break
            else:
                drop_nums.append(numerical_test)
                print("performance increased when", numerical_test, "dropped to", local_score, "from", best_score)
                best_score = local_score
                model.save_weights("best_model.h5")
                break

In [ ]:
print(drop_nums)

In [ ]:
numerical = [num for num in numerical_save if num not in drop_nums]

In [ ]:
model = nn_model()

In [ ]:
model.load_weights("best_model.h5")

## Checks on validation set


In [ ]:
X_valid = get_input_features(val_df)
X_test = get_input_features(test_df)
valid_preds = model.predict(X_valid, batch_size = 500, verbose = True)
score = roc_auc_score(y_valid, valid_preds)
print(score)

fine fitting on validation data

In [ ]:
hist = model.fit(X_valid,y_valid, batch_size = 8000, epochs = 3, verbose = 1)

# Prediction

In [ ]:
predictions = model.predict(X_test, batch_size = 2000, verbose = True)

In [ ]:
pd.DataFrame(valid_preds).describe()

In [ ]:
pd.DataFrame(predictions).describe()

In [ ]:
sample_submission['isFraud'] = predictions
sample_submission.to_csv('submission.csv')